# Trabalho prático 4: representação de dados em banco de dados relacional


## Criando a estrutura do banco


In [1]:
import sqlite3
import pandas as pd
caminho = './'
banco = 'banco.db'

conexao = sqlite3.connect(caminho + banco)

tabelas = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'", conexao)

existe_tabela_paises = False
existe_tabela_indicadores = False
existe_tabela_dados = False

for i, j in tabelas.iterrows():
    if(j['name'] == 'paises'):
        existe_tabela_paises = True
    if(j['name'] == 'indicadores'):
        existe_tabela_indicadores = True
    if(j['name'] == 'dados'):
        existe_tabela_dados = True

if not(existe_tabela_paises):
    script_cria_tabela_pais = open(
        caminho + 'cria_tabela_pais.sql', 'r').read()
    conexao.executescript(script_cria_tabela_pais)

if not(existe_tabela_indicadores):
    script_cria_tabela_indicadores = open(
        caminho + 'cria_tabela_indicadores.sql', 'r').read()
    conexao.executescript(script_cria_tabela_indicadores)

if not(existe_tabela_dados):
    script_cria_tabela_dados = open(
        caminho + 'cria_tabela_dados.sql', 'r').read()
    conexao.executescript(script_cria_tabela_dados)

conexao.commit()
conexao.close()


## Inserindo dados na tabela países


In [2]:
import pandas as pd

caminho: str = './'
arquivo: str = 'WDICountry.csv'
banco: str = 'banco.db'

df_paises = pd.read_csv(caminho + arquivo, encoding = 'utf_8', encoding_errors= 'surrogateescape')

script_insere_pais: list[str] = []

for coluna, linha in df_paises.iterrows():
    code = linha['Country Code']
    name = linha['Long Name']
    region = linha['Region']
    script = f"INSERT INTO paises (nome_curto, nome_longo, regiao) VALUES (\"{code}\",\"{name}\",\"{region}\");"
    script_insere_pais.append(script)

# gravando script em um arquivo
file_insert = open(caminho + 'script_insere_paises.sql', 'w')
file_insert.writelines(script_insere_pais)
file_insert.close()

# lendo script com os dados dos países
f = open(caminho + 'script_insere_paises.sql', "r").read()
conexao = sqlite3.connect(caminho + banco)

conexao.executescript(f)


tabelas = pd.read_sql_query("SELECT * FROM paises", conexao)

display(tabelas)

conexao.close()


,pais_id,nome_curto,nome_longo,regiao
0,1,ABW,Aruba,Latin America & Caribbean
1,2,AFG,Islamic State of Afghanistan,South Asia
2,3,AGO,People's Republic of Angola,Sub-Saharan Africa
3,4,ALB,Republic of Albania,Europe & Central Asia
4,5,AND,Principality of Andorra,Europe & Central Asia
...,...,...,...,...
258,259,XKX,Republic of Kosovo,Europe & Central Asia
259,260,YEM,Republic of Yemen,Middle East & North Africa
260,261,ZAF,Republic of South Africa,Sub-Saharan Africa
261,262,ZMB,Republic of Zambia,Sub-Saharan Africa
